So far we have used tf.keras mainly. That is usually enough for 95% of time, however if you need to dive deeper, you can use Tensorflow's lower level API.

# A quick tour of Tensorflow

Below are some characteristics of the TensorFlow API:

- Its core is very similar to Numpy, but with GPU support
- Supports distributed computing
- Its compiler allows for optimization of computing speed and memory usage. It works by extracting the *computation graph* from a Python function then optimizing it, and finally running it.
- Computation graphs are portable, meaning you can train models in one environment (e.g Python) and run it in another (e.g. Java or Android)
- Implements autodiff and provides excellent standard optimizers such as RMSPROP and Nadam

It also has features for loading and preprocessing data (tf.data, tf.io), image processing (tf.image), signal processing (tf.signal) and more.

TensorFlow is at the center of an extensive ecosystem of libraries. TensorBoard allows for vizualization, TensorFlow Extended (TFX) is a set of libraries built to productionize TensorFlow Projects, including tools for data validation, preprocessing, model analysis and serving. TensorFlow Hub provides a way to download and share pretrained neural networks. Finally you can check [TensorFlow Resources](https://www.tensorflow.org/resources) and this [github pages](https://github.com/jtoy/awesome-tensorflow) for more TensorFlow-based projects. 

# Using TensorFlow like Numpy

A tensor is usually a multidimensional array (just like numpy's ndarray), but it can also hold a scalar.

### Tensors and Operations

Use ```tf.constant()``` to create a tensor

In [3]:
import tensorflow as tf

In [7]:
t = tf.constant([[1., 2., 3.], [4., 5., 6.]])
t

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[1., 2., 3.],
       [4., 5., 6.]], dtype=float32)>

In [6]:
tf.constant(42)

<tf.Tensor: shape=(), dtype=int32, numpy=42>

In [8]:
t.shape

TensorShape([2, 3])

In [10]:
t.dtype

tf.float32

Indexing is the same as Numpy

In [11]:
t[:, 1:]

<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[2., 3.],
       [5., 6.]], dtype=float32)>

In [12]:
t[..., 1, tf.newaxis]

<tf.Tensor: shape=(2, 1), dtype=float32, numpy=
array([[2.],
       [5.]], dtype=float32)>

In [16]:
tf.square(t)

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[ 1.,  4.,  9.],
       [16., 25., 36.]], dtype=float32)>

In [17]:
t + 10

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[11., 12., 13.],
       [14., 15., 16.]], dtype=float32)>

Use the @ to perform matrix multiplication

In [20]:
t @ tf.transpose(t)

<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[14., 32.],
       [32., 77.]], dtype=float32)>

In [22]:

tf.linalg.matmul(t, tf.transpose(t))

<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[14., 32.],
       [32., 77.]], dtype=float32)>

You will find all basic math operations and most numpy operations in TensorFlow. Some might have a different name (e.g. np.mean() = tf.reduce_mean()). When the name differs, there is a good reason for it. For example for tf.reduce_sum(), the GPU implementation does not guarantee the order in which elements are added, so for 32-bit floats the result may change ever so slightly.

### Tensors and NumPy

You can create a tensor from a numpu array and vice versa. You can even apply TF operations to a numpy array and numpy operations to tensors

In [23]:
import numpy as np

a = np.array([2., 4., 5.])
tf.constant(a)

<tf.Tensor: shape=(3,), dtype=float64, numpy=array([2., 4., 5.])>

In [24]:
t.numpy()

array([[1., 2., 3.],
       [4., 5., 6.]], dtype=float32)

In [25]:
tf.square(a)

<tf.Tensor: shape=(3,), dtype=float64, numpy=array([ 4., 16., 25.])>

In [26]:
np.square(t)

array([[ 1.,  4.,  9.],
       [16., 25., 36.]], dtype=float32)

Note that NumPy uses 64-bit precision by default while TensorFlow uses 32-bit. This is because 32-bit is generally more than enough for Neural Nets and it runs faster while using less RAM. When you create a tensor from a NumPy array, set ```dtype=float32```

### Type conversions

TensorFlow does not execute automatic type conversions. It raises an exception if you try to execute an operation on tensors with incompatible types. For example you cannot add a float tensor and an integer tensor, nor add a 32-bit float and a 64-bit float.

In [27]:
tf.constant(2.) + tf.constant(40)

InvalidArgumentError: cannot compute AddV2 as input #1(zero-based) was expected to be a float tensor but is a int32 tensor [Op:AddV2]

You can use tf.cast if you really need to convert types

In [28]:
tf.constant(2.) + tf.cast(tf.constant(40), tf.float32)

<tf.Tensor: shape=(), dtype=float32, numpy=42.0>

### Variables

```tf.Tensor``` values we have seen so far are immutable. Thus we cannot use regular tensors to implement weights in a NN since they need to be tweaked by backpropagation. For this we can use ```tf.Variable```

In [29]:
v = tf.Variable([[1., 2., 3.], [4., 5., 6.]])
v

<tf.Variable 'Variable:0' shape=(2, 3) dtype=float32, numpy=
array([[1., 2., 3.],
       [4., 5., 6.]], dtype=float32)>

Variables can be modified using the ```assign()``` method

In [30]:
v.assign(2 + v)

<tf.Variable 'UnreadVariable' shape=(2, 3) dtype=float32, numpy=
array([[3., 4., 5.],
       [6., 7., 8.]], dtype=float32)>

In [31]:
v

<tf.Variable 'Variable:0' shape=(2, 3) dtype=float32, numpy=
array([[3., 4., 5.],
       [6., 7., 8.]], dtype=float32)>

### Other Data Structures

Some of the other structures available are

*Sparse tensors* (tf.SparseTensor)

Efficiently represent tensors containing mostly zeros. tf.sparse contains operations for these tensors

*Tensor Arrays* (tf.TensorArray)

Lists of tensors. Have fixed size by default but can optionally be made dynamic. All tensors they contain must have the same shape and data type

*Ragged tensors* (tf.RaggedTensor)

Static lists of tensors, where every tensor has the same shape and data type. 

*String tensors*

Regular tensors of type tf.string. These are byte strings, not unicode strings. Creating a tensor using a Unicode string automatically converts them to UTF-8. 

*Sets*

Are represented as regular (or sparse) tensors. For example ```tf.constant([[1,2], [3,4]])``` represents the two sets {1,2} and {3,4}. 

*Queues*

Store tensors across multiple steps. TensorFlow offers various kinds of queues: FIFO, PriorityQueues, RandomShuffleQueue) and batch items of different shapes by padding (PaddingFIFOQueue)

## Customizing Models and Training Algorithms

### Custom loss functions